In [1]:
%load_ext Cython

from enum import Enum
import copy

from hearthstone.enums import *
from fireplace import cards
from fireplace.game import Game
from fireplace.player import Player
import fireplace.cards

In [2]:
from enum import Enum
import copy

from hearthstone.enums import *
from fireplace import cards
from fireplace.game import Game
from fireplace.player import Player
import fireplace.cards
import random

In [3]:
%%cython

from enum import Enum
import copy

from hearthstone.enums import *
from fireplace import cards
from fireplace.game import Game
from fireplace.player import Player
import fireplace.cards

hunter_simple_deck = ["DS1_175", "DS1_185", "EX1_506", "CS2_196", "CS2_122", "CFM_315", "CS2_186", "CS2_182", "CS2_201", "CS2_172", "CS2_200", "CS2_118", "CS2_168", "CS2_119", "CS2_120", "DS1_175", "DS1_185", "EX1_506", "CS2_196", "CS2_122", "CS2_186", "CFM_315", "CS2_182", "CS2_201", "CS2_172", "CS2_200", "CS2_118", "CS2_168", "CS2_119", "CS2_120"]

import random

def minimum(input_list, attr):
	#value = min(input_list, key = lambda t:t[1].__dict__[attr]) 
	value = min([x[1].__dict__[attr] for x in input_list])
	return [x for x in input_list if x[1].__dict__[attr] == value]

def maximum(input_list, attr):
	#value = max(input_list, key = lambda t:t[1].__dict__[attr])
	value = max([x[1].__dict__[attr] for x in input_list])
	return [x for x in input_list if x[1].__dict__[attr] == value]

def random(input_list, attr):
	value = max([x[1].__dict__[attr] for x in input_list])
	return [random.choice([x for x in input_list if x[1].__dict__[attr] == value])]

def fullrandom(input_list, attr):
	return [random.choice(input_list)]

def condition(statement_list, gamestate):
	flag_condition = True
	for (argument, operator, value) in statement_list:
		arg = argument
		val = value
		if isinstance(argument, str):
			arg = gamestate.__dict__[argument]
		if isinstance(value, str):
			val = gamestate.__dict__[value]

		if operator(arg, value):
			pass
		else:
			flag_condition = False
			break
	
	return flag_condition

class Actions(Enum):
	PLAY = 1
	ATTACK = 2
	POWER = 3
	CONDITION = 4

class Effects(Enum):
	SUMMON = 1
	HIT = 2
	CUSTOM = 3

class ProtoCharacter:
	def __init__(self, id, atk, health, race, power):
		self.id = id
		self.atk = atk
		self.health = health
		self.race = race
		self.power = power

class CharacterState:
	def __init__(self, _char):
		self.id = _char.id
		self.atk = _char.atk
		self.health = _char.health
		self.race = _char.race
		if 'data' in _char.__dict__:
			self.power = self.buildPowerDict(_char.data.scripts)
		else:
			self.power = {'battlecry': [], 'update': []}
	
	def __str__(self):
		return str(self.id) + ": ATK/" + str(self.atk) + " HP/" + str(self.health) + " RACE/" + str(self.race) + " POWER/" + str(self.power)
	
	def buildPowerDict(self, charscript):
		result = {'battlecry': [], 'update': []}
		
		for data in charscript.play:
			if isinstance(data, fireplace.actions.Summon):
				temp_id = data._args[1]
				temp_tags = cards.db[temp_id].tags
				temp_char = ProtoCharacter(temp_id, temp_tags[GameTag.ATK], temp_tags[GameTag.HEALTH], temp_tags[GameTag.CARDRACE],[])
				result['battlecry'].append((Effects.SUMMON, temp_char))
			
			if isinstance(data, fireplace.actions.Hit):
				result['battlecry'].append((Effects.HIT, True, data._args[1]))
			
		for data in charscript.update:
			if data.buff == 'CS2_122e':
				result['update'].append((Effects.CUSTOM, 'Raid Leader'))
			
			if data.buff == 'DS1_175o':
				result['update'].append((Effects.CUSTOM, 'Timber Wolf'))

		return result

class GameState:
	def __init__(self, game):
		player = game.current_player
		opponent = player.opponent
		self.herohealth = player.hero.health
		self.mana = player.mana
		self.potential_damage = self.calculatePotentialDamage(player.characters)
		self.minions = [CharacterState(x) for x in player.characters]
		self.number_of_minions = len(player.characters) - 1 # minus 1 to remove the hero
		self.minion_damage = sum([x.atk for x in self.minions])
		self.enemy_herohealth = opponent.hero.health
		self.enemy_minions = [CharacterState(x) for x in opponent.characters]
		self.enemy_number_of_minions = len(opponent.characters) - 1 # minus 1 to remove the hero
		self.enemy_minion_damage = sum([x.atk for x in self.enemy_minions])
		self.updateEffects = self.minionsUpdateEffects()
	
	def __str__(self):
		temp = ""
		temp += "Hero Health: " + str(self.herohealth)
		temp += " Hero Mana: " + str(self.mana)
		temp += " Potential Damage: " + str(self.potential_damage)
		temp += " Minions: " + str(self.minions)
		temp += " Enemy Hero Health: " + str(self.enemy_herohealth)
		temp += " Enemy # of Minions: " + str(self.enemy_number_of_minions)
		
		return temp

	def copy(self, state):
		self.herohealth = state.herohealth
		self.mana = state.mana
		self.potential_damage = state.potential_damage
		self.minions = list(state.minions)
		self.number_of_minions = state.number_of_minions
		self.minion_damage = sum([x.atk for x in self.minions])
		self.enemy_herohealth = state.enemy_herohealth
		self.enemy_number_of_minions = state.enemy_number_of_minions
		self.enemy_minion_damage = sum([x.atk for x in self.enemy_minions])
		self.updateEffects = self.minionsUpdateEffects()

	def calculatePotentialDamage(self, characters):
		total = 0

		for character in characters:
			if character.can_attack():
				total += character.atk

		return total

	def updateMinionDamage(self):
		self.minion_damage = sum([x.atk for x in self.minions])
		self.enemy_minion_damage = sum([x.atk for x in self.enemy_minions])

	def minionsUpdateEffects(self):
		result = []

		for minion in self.minions:
			if len(minion.power['update']) > 0:
				for minion_power in minion.power['update']:
					result.append((minion, minion_power[0], minion_power[1]))

		return result

	def activatePower(self, charstate, type):
		for power, effect in charstate.power[type]:
			if power == Effects.SUMMON:
				self.addMinion(effect)

			elif power == Effects.HIT:
				pass

			elif power == Effects.CUSTOM:
				if effect == 'Raid Leader':
					for minion in self.minions:
						if id(minion) != id(charstate):
							minion.atk = minion.atk + 1

				elif effect == 'Timber Wolf':
					for minion in self.minions:
						if id(minion) != id(charstate):
							if minion.race == charstate.race:
								minion.atk = minion.atk + 1

	def addMinion(self, card):
		charstate = CharacterState(card)
		self.minions.append(charstate)
		if len(charstate.power['battlecry']) > 0:
			self.activatePower(charstate, 'battlecry')
		if len(charstate.power['update']) > 0:
			self.activatePower(charstate, 'update')
		self.number_of_minions = self.number_of_minions + 1
		self.updateMinionDamage()
		#self.potential_damage = self.calculatePotentialDamage(player.characters)

	def playSpell(self, card, target):
		spellscript = card.data.scripts

		flagHero = True

		for data in spellscript.play:
			if isinstance(data, fireplace.actions.Hit):
				if 'HERO' not in target.id:
					#print(target)
					for minion in self.enemy_minions:
						if minion.id == target.id and minion.atk == target.atk and minion.health == target.health:
							#print('here')
							minion.health = minion.health - data._args[1]
							flagHero = False

					if flagHero:
						for minion in self.minions:
							if minion.id == target.id and minion.atk == target.atk and minion.health == target.health:
								#print('here 2')
								minion.health = minion.health - data._args[1]
								flagHero = False
	
				else:
					#ADD SELF HERO HIT ??????????????????????
					self.enemy_herohealth = self.enemy_herohealth - data._args[1]
	
	def removeEffect(self, minion, opponent=False):
		if opponent:
			source = self.enemy_minions
		else:
			source = self.minions
	
		if not opponent:
			for (effect_minion, minion_power_type, minion_power) in self.updateEffects:
				if id(effect_minion) == id(minion):
					if minion_power_type == Effects.CUSTOM:
						if minion_power == 'Raid Leader':
							for target_minion in source:
								target_minion.atk = target_minion.atk - 1
						
						if minion_power == 'Timber Wolf':
							for target_minion in source:
								if target_minion.race == minion.race:
									target_minion.atk = target_minion.atk - 1
				
					break
		else:
			if len(minion.power['update']) > 0:
				for power_type, power in minion.power['update']:
					if power_type == Effects.CUSTOM:
						if power == 'Raid Leader':
							for target_minion in source:
								target_minion.atk = target_minion.atk - 1
						
						if power == 'Timber Wolf':
							for target_minion in source:
								if target_minion.race == minion.race:
									target.minion.atk = target_minion.atk - 1
	
	def minionAtk(self, character, target):
		attacker = None
		defender = None
		
		for char in self.minions:
			if char.id == character.id and char.atk == character.atk and char.health == character.health:
				attacker = char
				break
		
		for char in self.enemy_minions:
			if char.id == target.id and char.atk == target.atk and char.health == target.health:
				defender = char
				break
		
		if attacker != None and defender != None:
			if 'HERO' in defender.id:
				if target.controller != character.controller:
					self.minionDamage(attacker, defender, hero=True, self_hero=False)
				else:
					self.minionDamage(attacker, defender, hero=True, self_hero=True)
			else:
				self.minionDamage(attacker, defender)
	
			return True
		
		return False
		
	def minionDamage(self, attacker, defender, hero=False, self_hero=False):
		if hero:
			if self_hero:
				self.herohealth = self.herohealth - attacker.atk
			else:
				self.enemy_herohealth = self.enemy_herohealth - attacker.atk
		else:
			defender.health = defender.health - attacker.atk
			attacker.health = attacker.health - defender.atk
	
		self.updateState()
	
	def updateState(self):
		list_of_minions_to_remove = []
	
		for minion in self.minions:
			if minion.health <= 0:
				list_of_minions_to_remove.append(minion)

		for minion in list_of_minions_to_remove:
			self.minions.remove(minion)
			self.removeEffect(minion)

		list_of_minions_to_remove = []
	
		for minion in self.enemy_minions:
			if minion.health <= 0:
				list_of_minions_to_remove.append(minion)

		for minion in list_of_minions_to_remove:
			self.enemy_minions.remove(minion)
			#self.removeEffect(minion)

		self.number_of_minions = len(self.minions) - 1
		self.enemy_number_of_minions = len(self.enemy_minions) - 1
		self.updateMinionDamage()

PLAYER_1_NAME = "one"
PLAYER_2_NAME = "two"

class Test:
	def __init__(self):
		self.player1 = None
		self.hero1 = None
		self.deck1 = []

		self.player2 = None
		self.hero2 = None
		self.deck2 = []

		self.game = None

	def start(self):
		self.hero1 = CardClass.HUNTER.default_hero
		self.hero2 = CardClass.HUNTER.default_hero
		self.deck1 = list(hunter_simple_deck)
		self.deck2 = list(hunter_simple_deck)
		self.player1 = Player(PLAYER_1_NAME, self.deck1, self.hero1)
		self.player2 = Player(PLAYER_2_NAME, self.deck2, self.hero2)
		self.game = Game(players=(self.player1, self.player2))
		self.game.start()
		self.skipMulligan()

	def skipMulligan(self):
		self.player1.choice.choose()
		self.player2.choice.choose()

	def possibleNextAtkLight(self, used=[]):
		result = []
		player = self.game.current_player

		for x in range(0, len(player.characters)):
			if x not in used:
				character = player.characters[x]
				if character.can_attack():
					result.append((character, Actions.ATTACK))

		return result

	def possibleNextAtk(self):
		result = []
		player = self.game.current_player

		for character in player.characters:
			if character.can_attack():
				result.append((character, Actions.ATTACK))

		return result

	def simulatePossibleAtksLight(self, cards_atk=[], gstate=None, cards_used=[]):
		if gstate == None:
			gstate = GameState(self.game)

		result = []

		list_of_next_atks = self.possibleNextAtkLight(cards_used)

		for (character, type_of_action) in list_of_next_atks:
			p = self.game.current_player
			opp = p.opponent

			for target in character.targets:
				temp_state = GameState(self.game)
				temp_state.copy(gstate)

				#target_dict = {'card': target, 'atk': target.atk, 'health': target.health, 'opponent': True if target.controller.first_player != p.first_player else False}

				value = temp_state.minionAtk(character, target)

				if value:
					result.append((cards_atk + [(character, target)], temp_state))
					result.extend(self.simulatePossibleAtksLight(cards_atk + [(character, target)], temp_state, cards_used + [p.characters.index(character)]))

		return result

	def simulatePossibleAtks(self, cards_atk=[]):
		result = []

		list_of_next_atks = self.possibleNextAtk()

		for i in range(0, len(list_of_next_atks)):
			p = self.game.current_player
			opp = p.opponent

			index_of_char = p.characters.index(list_of_next_atks[i][0])

			list_of_targets = list_of_next_atks[i][0].targets

			for j in range(0, len(list_of_targets)):
				copy_test = Test()
				copy_test.game = copy.deepcopy(self.game)

				target = list_of_targets[j]

				target_dict = {'card': target, 'atk': target.atk, 'health': target.health, 'opponent': True if target.controller.first_player != p.first_player else False}

				if target_dict['opponent'] == True:
					index_of_target = opp.characters.index(target)
					copy_test.game.current_player.characters[index_of_char].attack(target=copy_test.game.current_player.opponent.characters[index_of_target])
				else:
					index_of_target = p.characters.index(target)
					copy_test.game.current_player.characters[index_of_char].attack(target=copy_test.game.current_player.characters[index_of_target])

				player = copy_test.game.current_player
				opponent = player.opponent
				result.append((cards_atk + [(p.characters[index_of_char], target_dict)], GameState(player.hero.health, player.mana, player.characters, opponent.hero.health, opponent.characters)))
				result.extend(copy_test.simulatePossibleAtks(cards_atk + [(p.characters[index_of_char], target_dict)]))

		return result

	def possibleNextActionLight(self, used=[], field_size=0):
		result = []
		player = self.game.current_player

		if field_size < 7:
			for x in range(0, len(player.hand)):
				if x not in used:
					card = player.hand[x]
					if card.is_playable():
						result.append((x, Actions.PLAY))
		
		if player.hero.power.is_usable() and player.hero.power not in used:
			result.append((player.hero.power, Actions.POWER))
		
		return result	
	
	def possibleNextAction(self):
		result = []
		player = self.game.current_player

		for card in player.hand:
			if card.is_playable():
				result.append((card, Actions.PLAY))

		if player.hero.power.is_usable():
			result.append((player.hero.power, Actions.POWER))

		return result
	
	def simulatePossibleActionsLight(self, cards_played=[], gstate=None, cards_used=[]):
		if gstate == None:
			gstate = GameState(self.game)
		
		current_mana = self.game.current_player.mana
		
		result = []
		
		list_of_possible_actions = self.possibleNextActionLight(cards_used, field_size=gstate.number_of_minions)
		
		for (card_index, type_of_action) in list_of_possible_actions:
			if (type_of_action == Actions.PLAY):
				card = self.game.current_player.hand[card_index]
				if isinstance(card, fireplace.card.Minion):
					temp_state = GameState(self.game)
					temp_state.copy(gstate)
					temp_state.addMinion(card)
					
					self.game.current_player.__dict__['_max_mana'] = self.game.current_player.__dict__['_max_mana'] - card.cost
					
					result.append((cards_played + [card], temp_state))
					result.extend(self.simulatePossibleActionsLight(cards_played + [card], temp_state, cards_used + [card_index]))
		
					self.game.current_player.__dict__['_max_mana'] = current_mana

				if isinstance(card, fireplace.card.Spell):
					if len(card.targets) > 0:
						for target in card.targets:
							temp_state = GameState(self.game)
							temp_state.copy(gstate)

							if card.id == 'GAME_005': #THE COIN							
								self.game.current_player.__dict__['_max_mana'] = self.game.current_player.__dict__['_max_mana'] + 1

								temp_state.mana = self.game.current_player.mana
								
								result.append((cards_played + [card], temp_state))
								result.extend(self.simulatePossibleActionsLight(cards_played + [card], temp_state, cards_used + [card_index]))
					
								self.game.current_player.__dict__['_max_mana'] = current_mana

							else:
								self.game.current_player.__dict__['_max_mana'] = self.game.current_player.__dict__['_max_mana'] - card.cost

								temp_state.mana = self.game.current_player.mana
								#target_dict = {'card': target, 'atk': target.atk, 'health': target.health, 'opponent': True if target.controller.first_player != self.game.current_player.first_player else False}

								temp_state.playSpell(card, target)
								temp_state.updateState()
								
								result.append((cards_played + [(card, target)], temp_state))
								result.extend(self.simulatePossibleActionsLight(cards_played + [(card, target)], temp_state, cards_used + [card_index]))
					
								self.game.current_player.__dict__['_max_mana'] = current_mana

			if (type_of_action == Actions.POWER):
				temp_state = GameState(self.game)
				temp_state.copy(gstate)
				if self.game.current_player.hero.power.id == 'DS1h_292':
					temp_state.enemy_herohealth = temp_state.enemy_herohealth - 2
					
				self.game.current_player.__dict__['_max_mana'] = self.game.current_player.__dict__['_max_mana'] - self.game.current_player.hero.power.cost
	
				temp_state.mana = self.game.current_player.mana

				result.append((cards_played + [card_index], temp_state))
				result.extend(self.simulatePossibleActionsLight(cards_played + [card_index], temp_state, cards_used + [card_index]))

				self.game.current_player.__dict__['_max_mana'] = self.game.current_player.__dict__['_max_mana'] + self.game.current_player.hero.power.cost
		
		self.game.current_player.__dict__['_max_mana'] = current_mana
		
		return result

	def simulatePossibleActions(self, cards_played=[]):
		result = []

		list_of_next_actions = self.possibleNextAction()

		for i in range(0, len(list_of_next_actions)):
			p = self.game.current_player
			opp = p.opponent
			card = list_of_next_actions[i][0]
			type_of_action = list_of_next_actions[i][1]
			target_flag = False

			if type_of_action == Actions.PLAY:
				index_of_card = p.hand.index(list_of_next_actions[i][0])

				target_flag = False
				if len(card.targets) > 0:
					target_flag = True
					list_of_targets = card.targets

					for j in range(0, len(list_of_targets)):
						copy_test = Test()
						copy_test.game = copy.deepcopy(self.game)

						target = list_of_targets[j]
						target_dict = {'card': target, 'atk': target.atk, 'health': target.health, 'opponent': True if target.controller.first_player != p.first_player else False}

						if target_dict['opponent'] == True:
							index_of_target = opp.characters.index(target)
							copy_test.game.current_player.hand[index_of_card].play(target=copy_test.game.current_player.opponent.characters[index_of_target])
						else:
							index_of_target = p.characters.index(target)
							copy_test.game.current_player.hand[index_of_card].play(target=copy_test.game.current_player.characters[index_of_target])

						player = copy_test.game.current_player
						opponent = player.opponent
						result.append((cards_played + [(card, target_dict)], GameState(player.hero.health, player.mana, player.characters, opponent.hero.health, opponent.characters)))
						result.extend(copy_test.simulatePossibleAtks(cards_played + [(card, target_dict)]))

				else:
					copy_test = Test()
					copy_test.game = copy.deepcopy(self.game)

					temp_list_of_next_actions = copy_test.possibleNextAction()

					temp_list_of_next_actions[i][0].play()


			elif type_of_action == Actions.POWER:
				copy_test = Test()
				copy_test.game = copy.deepcopy(self.game)

				copy_test.game.current_player.hero.power.use()

			if not target_flag:
				player = copy_test.game.current_player
				opponent = player.opponent

				result.append((cards_played + [card], GameState(player.hero.health, player.mana, player.characters, opponent.hero.health, opponent.characters)))
				result.extend(copy_test.simulatePossibleActions(cards_played + [card]))

		return result

class AI:
	pass

class HeuristicAI:
	def __init__(self, heuristic, tree=False):
		self.tree_flag = tree
		if not tree:
			self.heuristic = self.interpreter(heuristic)
		else:
			self.heuristic = heuristic

	def interpreter(self, heuristic):
		result = []

		for (action, item, param) in heuristic:
			if item == "min":
				result.append((action, minimum, param))
			elif item == "max":
				result.append((action, maximum, param))
			elif item == "condition":
				result.append((action, condition, param))

		return result

	def move(self, test):
		#list_of_actions = test.simulatePossibleActions()
		#list_of_atks = test.simulatePossibleAtks()
		list_of_actions = test.simulatePossibleActionsLight()
		#print (list_of_actions)
		#print(test.game.current_player.characters)
		#print(test.game.current_player.opponent.characters)
		list_of_atks = []
		#list_of_atks = test.simulatePossibleAtksLight()
		#print (list_of_atks)

		if self.tree_flag:
			self.treeHeuristic([list_of_actions, list_of_atks], test)
		else:       
			flag_condition_met = True
			for (action, function, param) in self.heuristic:
				if test.game.ended:
					break

				if action == Actions.CONDITION:
					flag_condition_met = function(param, GameState(test.game))
					continue
			
				if flag_condition_met:
					if action == Actions.PLAY or action == Actions.POWER or action == Actions.ATTACK:
						self.action(action, function, param, list_of_actions, list_of_atks, test)
			
				flag_condition_met = True

	def treeHeuristic(self, moves, test):
		global_gstate = GameState(test.game)
        
		moves_to_execute = self.heuristic(moves)
		type_of_action1 = moves_to_execute[0]
		#type_of_action2 = moves_to_execute[1]
        
		if len(type_of_action1) > 0:
			self.executeAction(type_of_action1, test, "summon")

		#RECALCULATING THE POSSIBLE ATK OPTIONS
		list_of_atks = test.simulatePossibleAtksLight()
		moves_to_execute = self.heuristic(([],list_of_atks))
		type_of_action2 = moves_to_execute[1]
		if len(type_of_action2) > 0:
			self.executeAction(type_of_action2, test, "attack")

	def executeAction(self, action_list, test, type_of_action):        
		if len(action_list) == 0:
			return
		elif len(action_list) == 1:
			#print(action_list[0][0])
			if type_of_action == "summon":
				#print([id(x) if not isinstance(x, tuple) else id(x[0]) for x in action_list[0][0]])
				#print([x.is_playable() if not isinstance(x, tuple) else x[0].is_playable() for x in action_list[0][0]])
				self.play(action_list[0][0], test.game)
			else:
				#print([id(x[0]) for x in action_list[0][0]])
				#print([x[0].can_attack() for x in action_list[0][0]])
				self.attack(action_list[0][0], test.game)
		else:
			if type_of_action == "summon":
				temp = maximum(action_list, "mana")
				#print(temp[0][0])
				#print([id(x) if not isinstance(x, tuple) else id(x[0]) for x in action_list[0][0]])
				#print([x.is_playable() if not isinstance(x, tuple) else x[0].is_playable() for x in temp[0][0]])
				self.play(temp[0][0], test.game)
			else:
				#print(action_list[0][0])
				#print([id(x[0]) for x in action_list[0][0]])
				#print([x[0].can_attack() for x in action_list[0][0]])
				self.attack(action_list[0][0], test.game)
        
	def action(self, act, function, param, list_of_actions, list_of_atks, test):
		if act == Actions.PLAY or act == Actions.POWER:
			if len(list_of_actions) > 0:
				move = function(list_of_actions, param)
				if len(move) == 1:
					self.play(move[0][0], test.game)
				else:
					temp = heuristicfunctions.maximum(move, "mana")
					self.play(temp[0][0], test.game)

		elif act == Actions.ATTACK:
			if len(list_of_atks) > 0:
				move = function(list_of_atks, param)
				if len(move) == 1:
					self.attack(move[0][0], test.game)
				else:
					self.attack(move[0][0], test.game)

	def play(self, move, game):
		for action in move:
			if game.ended:
				break

			if (isinstance(action, fireplace.card.HeroPower)):
				game.current_player.hero.power.use()
			else:
				if not isinstance(action, tuple):
					action.play()
				else:
					action[0].play(target=action[1])
	
	def attack(self, move, game):
		for (atk_char, target) in move:
			if game.ended:
				break

			#print(game.ended)
			atk_char.attack(target=target)

class GameHandler:
	def __init__(self, test_case, players):
		self.game_tester = test_case
		self.players = players

	def run(self):
		self.game_tester.start()

		current_player = 0
		
		while not self.game_tester.game.ended:
			#if current_player > 0:
			#	print("RANDOM")
			#else:
			#	print("HEURISTIC")
			#print(self.game_tester.game.current_player.mana)
			#print("HAND:" + str(self.game_tester.game.current_player.hand))
			#print("HAND:" + str([id(x) for x in self.game_tester.game.current_player.hand]))
			#print("FIELD:" + str(self.game_tester.game.current_player.characters))
			#print("FIELD:" + str([id(x) for x in self.game_tester.game.current_player.characters]))
			#print(self.game_tester.game.current_player.hand)
			try:
				self.players[current_player].move(self.game_tester)
				self.game_tester.game.end_turn()
			except:
				pass
				#print("Unexpected error:", sys.exc_info()[0])
				#raise
			#except:
			#	print("exception")
			#	break
				#print("Exception: ")
				#print(self.game_tester.game.current_player.hero.health)
				#print(self.game_tester.game.current_player.first_player)
				#print(self.game_tester.game.current_player.opponent.hero.health)
			
			current_player = (current_player + 1) % len(self.players)

		return self.game_result(self.game_tester.game) 

	def game_result(self, game):
		result = {'order': [], 'number': [], 'score': 0} #order tells if the first player or the second player won ; number tells if player 1 or player 2 won

		if game.current_player.hero.health > 0 and game.current_player.opponent.hero.health > 0:
			print ("UNFINISHED GAME")
		if game.current_player.hero.health > 0:
			winner = game.current_player
			loser = winner.opponent
		else:
			loser = game.current_player
			winner = loser.opponent

		if winner.first_player:
			result['order'] = [1, 0]
			result['score'] = game.current_player.hero.health - game.current_player.opponent.hero.health
		else:
			result['order'] = [0, 1]
			result['score'] = game.current_player.opponent.hero.health - game.current_player.hero.health

		if winner.name == PLAYER_1_NAME:
			result['number'] = [1, 0]
		else:
			result['number'] = [0, 1]
        
		return result

def testrun():
	t.start()
	t.game.end_turn()
	for card in t.game.current_player.hand:
		if len(card.targets) == 0:
			if card.is_playable():
				card.play()
	t.game.end_turn()
	t.game.end_turn()
	for card in t.game.current_player.hand:
		if len(card.targets) == 0:
			if card.is_playable():
				card.play()
	t.game.end_turn()
	t.game.end_turn()
	t.game.end_turn()	

	h = t.simulatePossibleActionsLight()

#### STOP THE LOGGING!!!!

from fireplace.logging import log
import logging

log.setLevel(level=logging.CRITICAL)

#### STOP THE LOGGING!!!!

import time
import operator

p1_hai = HeuristicAI([(Actions.PLAY, "max", "minion_damage"), (Actions.ATTACK, "min", "enemy_herohealth")])
p2_hai = HeuristicAI([(Actions.PLAY, "max", "number_of_minions"), (Actions.ATTACK, "min", "enemy_number_of_minions")])
p22_hai = HeuristicAI([(Actions.PLAY, "max", "number_of_minions"), (Actions.CONDITION, "condition", [("enemy_herohealth", operator.le, "minion_damage")]), (Actions.ATTACK, "min", "enemy_herohealth"), (Actions.CONDITION, "condition", [("enemy_herohealth", operator.gt, "minion_damage")]), (Actions.ATTACK, "min", "enemy_number_of_minions")])
#hai2 = HeuristicAI([(Actions.PLAY, "fullrandom", "mana"), (Actions.ATTACK, "fullrandom", "enemy_number_of_minions")])
#hai2 = HeuristicAI([(Actions.PLAY, "random", "mana"), (Actions.ATTACK, "min", "enemy_herohealth")])

cards.db.initialize()

start = time.time()

def evaluate(p1_heuristic, p2_heuristic, n=100):
    hai = p1_heuristic
    hai2 = p2_heuristic
    
    result = {'p1': 0, 'p2': 0}
    for i in range(0, int(n/2)):
        temp = GameHandler(Test(), [hai, hai2])
        match = temp.run()

        result['p1'] += match['order'][0]
        result['p2'] += match['order'][1]
        #result['order'][0] += match['order'][0]
        #result['order'][1] += match['order'][1]
        #result['number'][0] += match['number'][0]
        #result['number'][1] += match['number'][1]

    for i in range(0, int(n/2)):
        temp = GameHandler(Test(), [hai2, hai])
        match = temp.run()

        result['p1'] += match['order'][1]
        result['p2'] += match['order'][0]
        #result['order'][0] += match['order'][0]
        #result['order'][1] += match['order'][1]
        #result['number'][0] += match['number'][0]
        #result['number'][1] += match['number'][1]
    
    return result['p1'] / (result['p1'] + result['p2'])

ev = evaluate(p1_hai, p2_hai, n=100)
print("Ellapsed Time: " + str(time.time() - start))
print(ev)
ev = evaluate(p1_hai, p22_hai, n=100)
print("Ellapsed Time: " + str(time.time() - start))
print(ev)

In [4]:
from deap import creator, gp, base, tools, algorithms

import operator
import random as randomizer

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMax)
creator.create("GameHeuristicIndividualMax", gp.PrimitiveTree, fitness=creator.FitnessMax)

global global_gstate

class MoveList(list):
    pass

class MoveListFinal(list):
    pass

def IfThenElse(input_condition, output1, output2):
    if input_condition:
        return output1
    else:
        return output2

global global_gstate
    
def opLessEquals(string, int):
    if global_gstate.__dict__[string] <= int:
        return True
    return False

def opLessEqualsStr(string, string2):
    if global_gstate.__dict__[string] <= gstate.__dict__[string2]:
        return True
    return False

def opGreaterEquals(string, int):
    if global_gstate.__dict__[string] >= int:
        return True
    return False

def opGreaterEqualsStr(string, string2):
    if global_gstate.__dict__[string] >= gstate.__dict__[string2]:
        return True
    return False

def minimum_attr(input_list, attr):
    if len(input_list) == 0:
        return []
    value = min([x[1].__dict__[attr] for x in input_list])
    return [x for x in input_list if x[1].__dict__[attr] == value]

def maximum_attr(input_list, attr):
    if len(input_list) == 0:
        return []
    value = max([x[1].__dict__[attr] for x in input_list])
    return [x for x in input_list if x[1].__dict__[attr] == value]

def minimum_minimum(input_list, attr1, attr2):
    return (minimum_attr(input_list[0], attr1), minimum_attr(input_list[1], attr2))

def minimum_maximum(input_list, attr1, attr2):
    return (minimum_attr(input_list[0], attr1), maximum_attr(input_list[1], attr2))

def maximum_minimum(input_list, attr1, attr2):
    return (maximum_attr(input_list[0], attr1), minimum_attr(input_list[1], attr2))

def maximum_maximum(input_list, attr1, attr2):
    return (maximum_attr(input_list[0], attr1), maximum_attr(input_list[1], attr2))

def randMove(input_list):
    if len(input_list) == 0:
        return []
    return [randomizer.choice(input_list)]

def randomMove(input_list):
    return (randMove(input_list[0]), randMove(input_list[1]))

pset = gp.PrimitiveSetTyped("MoveSelector", [MoveList], MoveListFinal)
pset.renameArguments(ARG0="Moves")

pset.addTerminal(True, bool)
pset.addTerminal(False, bool)
for i in range(0,11):
    pset.addTerminal(i, int)
pset.addTerminal("herohealth", str)
pset.addTerminal("mana", str)
pset.addTerminal("potential_damage", str)
pset.addTerminal("minions", str)
pset.addTerminal("number_of_minions", str)
pset.addTerminal("minion_damage", str)
pset.addTerminal("enemy_herohealth", str)
pset.addTerminal("enemy_minions", str)
pset.addTerminal("enemy_number_of_minions", str)
pset.addTerminal("enemy_minion_damage", str)
pset.addPrimitive(operator.and_, [bool, bool], bool)
#pset.addPrimitive(operator.eq, [int, int], bool)
#pset.addPrimitive(operator.ge, [int, int], bool)
pset.addPrimitive(opLessEquals, [str, int], bool)
pset.addPrimitive(opLessEqualsStr, [str, str], bool)
pset.addPrimitive(opGreaterEquals, [str, int], bool)
pset.addPrimitive(opGreaterEqualsStr, [str, str], bool)

pset.addPrimitive(IfThenElse, [bool, MoveListFinal, MoveListFinal], MoveListFinal)
#pset.addPrimitive(minimum_attr, [MoveList, str], MoveList)
#pset.addPrimitive(maximum_attr, [MoveList, str], MoveList)
pset.addPrimitive(minimum_minimum, [MoveList, str, str], MoveListFinal)
pset.addPrimitive(minimum_maximum, [MoveList, str, str], MoveListFinal)
pset.addPrimitive(maximum_minimum, [MoveList, str, str], MoveListFinal)
pset.addPrimitive(maximum_maximum, [MoveList, str, str], MoveListFinal)
pset.addPrimitive(randomMove, [MoveList], MoveListFinal)


In [5]:
possible_strings = ["herohealth",
                "mana",
                "potential_damage",
                #"minions",
                "number_of_minions",
                "minion_damage",
                "enemy_herohealth",
                #"enemy_minions",
                "enemy_number_of_minions",
                "enemy_minion_damage"
               ]
terminals = ['Moves']
terminal_funcs = []
terminal_funcs.extend(['minimum_minimum', 'minimum_maximum', 'maximum_minimum', 'maximum_maximum'])
#terminals.extend(['minimum_minimum' for x in range(0, len(possible_strings) * len(possible_strings))])
#terminals.extend(['minimum_maximum' for x in range(0, len(possible_strings) * len(possible_strings))])
#terminals.extend(['maximum_minimum' for x in range(0, len(possible_strings) * len(possible_strings))])
#terminals.extend(['maximum_maximum' for x in range(0, len(possible_strings) * len(possible_strings))])

possible_ints = range(0, 30+1)
possible_bools_operations = ["and_", "opLessEquals", "opLessEqualsStr", "opGreaterEquals", "opGreaterEqualsStr"]
#possible_bools_operations.extend(["opLessEquals" for x in range(0,len(possible_strings) * len(possible_strings))])
#possible_bools_operations.extend(["opLessEqualsStr" for x in range(0,len(possible_strings) * len(possible_strings))])
#possible_bools_operations.extend(["opGreaterEquals" for x in range(0,len(possible_strings) * len(possible_strings))])
#possible_bools_operations.extend(["opGreaterEqualsStr" for x in range(0,len(possible_strings) * len(possible_strings))])


In [6]:
import random
import multiprocessing

def generateBoolOp():
    operation = random.choice(possible_bools_operations)
    
    if operation == "and_":
        param1 = generateBoolOp()
        param2 = generateBoolOp()
    else:
        param1 = random.choice(possible_strings)
        if 'Str' in operation:
            param2 = random.choice(possible_strings)
        else:
            param2 = str(random.choice(possible_ints))
    
    return operation + "(" + param1 + "," + param2 + ")"

def generateTerminal():
    term = random.choice(terminal_funcs)

    #if term == "Moves":
    #    return term

    param1 = random.choice(possible_strings)
    param2 = random.choice(possible_strings)

    return term + "(Moves," + param1 + "," + param2 + ")"
    
def generateIndividualString(depth):
    if depth == 0:
        return generateTerminal()
    else:
        result = ""
        for i in range(0, depth):
            result += "IfThenElse(" + generateBoolOp() + "," + generateTerminal() + ","
        
        result += generateTerminal()
        for i in range(0, depth):
            result += ")"
        
        return result

def generateIndividual(depth, pset, type_=None):
    return gp.PrimitiveTree.from_string(generateIndividualString(depth), pset)

def individualForMutation(depth, pset, type_=None):
    result = ""
    if type_ == bool:
        result = generateBoolOp()
    if type_ == int:
        result = str(random.choice(possible_ints))
    if type_ == str:
        result = random.choice(possible_strings)
    if type_ == MoveList:
        result = terminals[0]
    if type_ == MoveListFinal:
        #result = generateIndividualString(depth)
        result = generateTerminal()
        
    return gp.PrimitiveTree.from_string(result, pset)

def mutateExpressionTree(indv, pset):
    #length = len(indv)
    mutable_elements = []
    
    for expr in indv:
        if expr.name != 'IfThenElse' and expr.name != "ARG0":
            mutable_elements.append(expr)
    
    chosen = random.choice(mutable_elements)
    #print("Chosen: " + chosen.name)
    new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)
    if chosen.ret != MoveList and chosen.ret != bool:
        while (chosen.name == new_gene[0].name):
            new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)
    #print("New: " + new_gene[0].name)
    
    index_for_mutation = indv.index(chosen)
    
    temp = []
    if chosen.name == "and_":
        subtree_slice = indv.searchSubtree(index_for_mutation)
        subtree = indv[subtree_slice]
        
        end_of_subtree = indv.index(subtree[len(subtree)-1])
        
        temp = indv[:index_for_mutation] + new_gene + indv[end_of_subtree+1:]
    elif isinstance(chosen, gp.Terminal):
        temp = indv[:index_for_mutation] + new_gene + indv[index_for_mutation+1:]
    elif isinstance(chosen, gp.Primitive):
        end_of_subtree = 0
        for i in range(1, len(indv)-index_for_mutation):
            if isinstance(indv[index_for_mutation+i], gp.Primitive):
                break
        if i < len(indv)-index_for_mutation-1:
            end_of_subtree = i - 1 + index_for_mutation
        else:
            end_of_subtree = i + index_for_mutation
    
        temp = indv[:index_for_mutation] + new_gene + indv[end_of_subtree+1:]
    
    #print("temp: " + str([x.name for x in temp]))
    
    #for i in range(0, len(new_gene)):
    #    indv[index_for_mutation+i] = new_gene[i]

    return gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(temp)), pset)

#TerminalTrue = gp.PrimitiveTree.from_string('IfThenElse(True, MoveList([]), MoveList([]))', pset)
#print(TerminalTrue)

toolbox = base.Toolbox()
toolbox.register("compile", gp.compile, pset=pset)
#toolbox.register("heuristic", gp.genGrow, pset=pset, min_=0, max_=2)
toolbox.register("heuristic", generateIndividual, depth=1, pset=pset)
toolbox.register("individual", tools.initIterate, creator.GameHeuristicIndividualMax, toolbox.heuristic)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("select", tools.selBest)
#toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selDoubleTournament, parsimony_size=1.7, fitness_first=True, fitness_size=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("heuristic_mut", individualForMutation, depth=1)
#toolbox.register("mutate", gp.mutUniform, expr=toolbox.heuristic_mut, pset=pset)
toolbox.register("mutate", mutateExpressionTree, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=3)) #Depth + 2 to allow ANDs
#toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=3)) #Depth + 2 to allow ANDs

opponent_heuristic = "randomMove(Moves)"
toolbox.register("opponent", gp.PrimitiveTree.from_string, string=opponent_heuristic, pset=pset)

#pool = multiprocessing.Pool()
#toolbox.register("map", pool.map)

#pop = toolbox.population(n=50)
#hof = tools.HallOfFame(3)
#pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=10*k, 
#                                     lambda_=30*k, cxpb=.5, mutpb=.1, ngen=ngen, 
#                                     stats=mstats, halloffame=hof, verbose=True)

#for i, item in enumerate(hof.items):
#    print((i,str(item)))
'''
ind1 = toolbox.individual()
ind2 = toolbox.individual()
print(ind1)
print(ind2)
print(ind1.height)
print(ind2.height)

mut = toolbox.mutate(ind1)
for i in mut:
    print(i)
    print(i.height)
'''

'\nind1 = toolbox.individual()\nind2 = toolbox.individual()\nprint(ind1)\nprint(ind2)\nprint(ind1.height)\nprint(ind2.height)\n\nmut = toolbox.mutate(ind1)\nfor i in mut:\n    print(i)\n    print(i.height)\n'

In [7]:
#### STOP THE LOGGING!!!!

from fireplace.logging import log
import logging

log.setLevel(level=logging.CRITICAL)

#### STOP THE LOGGING!!!!

import time
import operator

#p1_hai = HeuristicAI(toolbox.individual(), True)
#p1_hai = HeuristicAI(toolbox.compile(expr=gp.PrimitiveTree.from_string("maximum_minimum(Moves,'minion_damage', 'enemy_herohealth')", pset)), True)
#p2_hai = HeuristicAI(toolbox.compile(expr=gp.PrimitiveTree.from_string("randomMove(Moves)", pset)), True)

cards.db.initialize()

start = time.time()

def evaluateIndv(player, opponent, n=100):
    hai = HeuristicAI(toolbox.compile(expr=player), True)
    hai2 = HeuristicAI(toolbox.compile(expr=opponent), True)
    
    result = {'p1': 0, 'p2': 0, 'score_player': 0}
    for i in range(0, int(n/2)):
        temp = GameHandler(Test(), [hai, hai2])
        match = temp.run()

        result['p1'] += match['order'][0]
        result['p2'] += match['order'][1]
        result['score_player'] += match['score']
        #result['order'][0] += match['order'][0]
        #result['order'][1] += match['order'][1]
        #result['number'][0] += match['number'][0]
        #result['number'][1] += match['number'][1]

    for i in range(0, int(n/2)):
        temp = GameHandler(Test(), [hai2, hai])
        match = temp.run()

        result['p1'] += match['order'][1]
        result['p2'] += match['order'][0]
        result['score_player'] -= match['score']
        #result['order'][0] += match['order'][0]
        #result['order'][1] += match['order'][1]
        #result['number'][0] += match['number'][0]
        #result['number'][1] += match['number'][1]
    
    mean_score = (float(result['score_player']) / float(n))
    #print(mean_score)
    #print((float(result['p1']) / float(result['p1'] + result['p2'])))

    return ((float(result['p1']) / float(result['p1'] + result['p2'])) + (0.1 * mean_score),)
    #return (result['p1'] / (result['p1'] + result['p2']), )

#ev = evaluate(p1_hai, p2_hai, n=100)
#print("Ellapsed Time: " + str(time.time() - start))
#print(ev)


In [9]:
toolbox.register("evaluate", evaluateIndv, opponent=toolbox.opponent(), n=30)

ngen=2
k=3
pop = toolbox.population(n=10)
hof = tools.HallOfFame(3)
#pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=10*k,
#                                     lambda_=30*k, cxpb=.5, mutpb=.1, ngen=ngen,
#                                      halloffame=hof, verbose=True)
#pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=10*k,
#                                     lambda_=30*k, cxpb=.5, mutpb=.1, ngen=ngen,
#                                      stats=mstats, halloffame=hof, verbose=True)
#for i, item in enumerate(hof.items):
#    print((i,str(item)))

In [1]:
for ind in pop:
    fit = toolbox.evaluate(ind)
    ind.fitness = fit

NameError: name 'pop' is not defined

In [14]:
print (pop[9].fitness)

(0.43333333333333335,)


In [60]:
best = toolbox.select(pop, k=3)
#new_ind = toolbox.mate(best[0], best[1])
#print(new_ind)
children = mateBest(best)
print(children)
#print(toolbox.mutate(children[0]))

best 1 140459565201032
best 2 140459663435640
child 1 140459657883720
child 2 140459565401576
best 1 140459565201032
best 2 140459663437640
child 1 140459657886520
child 2 140459657885880
best 1 140459663435640
best 2 140459663437640
child 1 140459657885560
child 2 140459657885160
[[<deap.gp.Primitive object at 0x7fbf515e2818>, <deap.gp.Primitive object at 0x7fbf515e2778>, <deap.gp.Terminal object at 0x7fbf50023bd0>, <deap.gp.Terminal object at 0x7fbf50023ab0>, <deap.gp.Terminal object at 0x7fbf515e8090>, <deap.gp.Primitive object at 0x7fbf515e2958>, <deap.gp.Terminal object at 0x7fbf515e8090>, <deap.gp.Terminal object at 0x7fbf50023990>, <deap.gp.Terminal object at 0x7fbf50023b40>], [<deap.gp.Primitive object at 0x7fbf515e2818>, <deap.gp.Primitive object at 0x7fbf515e2638>, <deap.gp.Terminal object at 0x7fbf50023948>, <deap.gp.Terminal object at 0x7fbf50023b88>, <deap.gp.Terminal object at 0x7fbf515e8090>, <deap.gp.Primitive object at 0x7fbf515e28b8>, <deap.gp.Terminal object at 0x7fb

In [8]:
import random, itertools
import multiprocessing as mp

def findFitness(population):
    pop = population
    
    pool = mp.Pool(mp.cpu_count()-2)
    fitness_list = pool.map(toolbox.evaluate, pop)
    pool.terminate()
    
    for i in range(0, len(fitness_list)):
        pop[i].fitness = fitness_list[i]
    
    return pop

def mateBest(best):
    new_indv = []
    
    for i in range(0, len(best)-1):
        for j in range(i+1, len(best)):
            children = toolbox.mate(gp.PrimitiveTree.from_string(str(best[i]), pset), gp.PrimitiveTree.from_string(str(best[j]), pset))
            new_indv.extend(children)

    return new_indv

def mateElite(elite, size_of_children=60):
    children = []
    
    pairings = tuple(itertools.combinations(elite, 2))
    selected = random.sample(pairings, int(float(size_of_children) / 2.0))
    
    for i in range(0, len(selected)):
        children.extend(toolbox.mate(gp.PrimitiveTree.from_string(str(selected[i][0]), pset), gp.PrimitiveTree.from_string(str(selected[i][1]), pset)))

    return children
        
def mutateChildren(children, mut_prob):
    for child in children:
        rand_value = random.uniform(0.0, 1.0)
        if rand_value <= mut_prob:
            child = toolbox.mutate(child)
    
    return children

def createMutationIndv(indv):
    mutation = gp.PrimitiveTree.from_string(str(indv), pset)
    
    return toolbox.mutate(mutation, pset=pset)

def generation(population, mut_prob, k):
    pop = population
    pop_size = len(pop)

    elite_size = int(0.2 * pop_size)
    elite = toolbox.select(pop, elite_size)
    elite_mutations = []
    for i in range(0, len(elite)):
        elite_mutations.append(createMutationIndv(elite[i]))
        
    children = mateElite(elite, int(0.6 * pop_size))
    children = mutateChildren(children, mut_prob)

    #best = toolbox.select(pop, k)
    #children_no_fitness = mateBest(best)
    #children_no_fitness = mutateChildren(children_no_fitness, mut_prob)
    
    #children = findFitness(children_no_fitness)

    pop = elite + elite_mutations + children
    pop = findFitness(pop)
    #pop = toolbox.select(pop, pop_size)
    
    return pop

def gen0(population):
    pop = population
    pop = findFitness(pop)
    
    return pop

def createLogFile(filename, start_string):
    f = open(filename, 'w')
    f.write(start_string)
    f.close()

def logToFile(filename, input_string):
    f = open(filename, 'a')
    f.write(input_string)
    f.close()

In [ ]:
import time

toolbox.register("evaluate", evaluateIndv, opponent=toolbox.opponent(), n=250)

filename = "gen-output.txt"

ngen=30
k=10
mut_prob = 0.1
pop = toolbox.population(n=100)
#hof = tools.HallOfFame(3)

start = time.time()
createLogFile(filename, "Gen 0 Start\n")

pop = gen0(pop)

logToFile(filename, "Gen 0 End\nEllapsed Time: " + str(time.time() - start) + "\n")

for i in range(0, ngen):
    start = time.time()
    logToFile(filename, "Gen " + str(i+1) + "\n")
    
    pop = generation(pop, mut_prob, k)
    
    top = toolbox.select(pop, 1)
    logToFile(filename, "Top: " + str(top[0]) + "\n")
    logToFile(filename, "Fitnees: " + str(top[0].fitness) + "\n")
    
    logToFile(filename, "Ellapsed Time: " + str(time.time() - start) + "\n")
    

print("==================================")
print("==================================")
print("==================================")

best = toolbox.select(pop, 3)
for b in best:
    logToFile(filename, str(b) + "\n")
    logToFile(filename, str(b.fitness) + "\n")

In [12]:
best = toolbox.select(pop, 3)
for b in best:
    print(b)
    print(b.fitness)

maximum_minimum(minimum_maximum(Moves, 'enemy_number_of_minions', 'enemy_minion_damage'), 'minion_damage', 'enemy_herohealth')
(1.0,)
maximum_minimum(minimum_maximum(Moves, 'enemy_number_of_minions', 'number_of_minions'), 'minion_damage', 'enemy_herohealth')
(1.0,)
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
(1.0,)


maximum_minimum(minimum_maximum(Moves, 'enemy_number_of_minions', 'enemy_minion_damage'), 'minion_damage', 'enemy_herohealth')
(1.0,)
maximum_minimum(minimum_maximum(Moves, 'enemy_number_of_minions', 'number_of_minions'), 'minion_damage', 'enemy_herohealth')
(1.0,)
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
(1.0,)

minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'potential_damage', 'herohealth'), 'number_of_minions', 'enemy_minion_damage'), 'enemy_herohealth', 'minions')
(1.0,)
minimum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'minion_damage', 'minions'), 'enemy_herohealth', 'enemy_number_of_minions')
(1.0,)
minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'potential_damage', 'number_of_minions'), 'minion_damage', 'minions'), 'enemy_herohealth', 'number_of_minions')
(1.0,)

In [24]:
print(evaluateIndv(player=best[0], opponent=best[1], n=100))
print(evaluateIndv(player=best[0], opponent=best[2], n=100))
print(evaluateIndv(player=best[1], opponent=best[2], n=100))

(0.51,)
(0.42,)
(0.43,)


In [25]:
ranked = toolbox.select(pop, 100)

In [30]:
print(ranked[99])
print(ranked[99].fitness)
print(evaluateIndv(player=best[2], opponent=ranked[99], n=100))

minimum_minimum(maximum_minimum(Moves, 'herohealth', 'minions'), 'enemy_herohealth', 'enemy_number_of_minions')
(0.0,)
(1.0,)


In [31]:
print(ranked[50])
print(ranked[50].fitness)

minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'number_of_minions', 'enemy_herohealth'), 'herohealth', 'minions'), 'enemy_herohealth', 'enemy_number_of_minions')
(0.94,)


In [32]:
print(evaluateIndv(player=best[2], opponent=ranked[50], n=100))

(0.63,)


In [35]:
counter = 0
for i in pop:
    if i != best[2]:
        print(str(counter))
        print(evaluateIndv(player=best[2], opponent=i, n=100))
    counter += 1

0
(0.62,)
1
(0.61,)
2
(0.57,)
3
(0.43,)
4
(0.48,)
5
(0.52,)
6
(0.54,)
7
(0.54,)
8
(0.49,)
9
(0.61,)
10
(0.52,)
11
(0.55,)
12
(0.52,)
13
(0.5,)
14
(0.54,)
15
(0.47,)
16
(0.51,)
17
(0.52,)
18
(0.5,)
19
(0.48,)
20
(0.52,)
21
(0.53,)
22
(0.71,)
23
(0.91,)
24
(0.58,)
25
(0.67,)
26
(1.0,)
27
(0.98,)
28
(0.56,)
29
(0.53,)
30
(0.55,)
31
(0.46,)
32
(1.0,)
33
(0.51,)
34
(0.57,)
35
(0.57,)
36
(0.53,)
37
(0.65,)
38
(0.56,)
39
(0.52,)
40
(0.53,)
41
(0.47,)
42
(0.56,)
43
(0.5,)
44
(0.77,)
45
(0.46,)
46
(0.53,)
47
(0.52,)
48
(0.49,)
49
(0.47,)
50
(0.51,)
51
(1.0,)
52
(0.43,)
53
(0.57,)
54
(0.52,)
55
(0.51,)
56
(1.0,)
57
(0.5,)
58
(0.53,)
59
(0.45,)
60
(0.58,)
61
(0.48,)
62
(0.55,)
63
(0.51,)
64
(0.52,)
65
(1.0,)
66
(0.43,)
68
(0.7,)
69
(0.51,)
70
(0.67,)
71
(0.61,)
72
(0.48,)
73
(1.0,)
74
(0.56,)
75
(0.63,)
76
(0.55,)
77
(1.0,)
78
(0.62,)
79
(0.55,)
80
(1.0,)
81
(0.58,)
82
(0.73,)
83
(0.56,)
84
(0.95,)
85
(0.55,)
86
(0.53,)
87
(0.98,)
88
(0.4,)
89
(0.64,)
90
(0.74,)
91
(0.52,)
92
(1.0,)
93
(0.61,)
94

In [36]:
for i in pop:
    print(i)

minimum_minimum(maximum_minimum(maximum_minimum(maximum_minimum(Moves, 'number_of_minions', 'enemy_herohealth'), 'number_of_minions', 'number_of_minions'), 'herohealth', 'number_of_minions'), 'enemy_herohealth', 'enemy_number_of_minions')
minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'number_of_minions', 'number_of_minions'), 'potential_damage', 'enemy_number_of_minions'), 'enemy_herohealth', 'potential_damage')
minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'potential_damage', 'number_of_minions'), 'minion_damage', 'minions'), 'enemy_herohealth', 'number_of_minions')
minimum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of_minions', 'enemy_herohealth'), 'number_of_minions', 'number_of_minions'), 'enemy_herohealth', 'enemy_number_of_minions')
minimum_minimum(maximum_minimum(maximum_minimum(minimum_minimum(Moves, 'enemy_number_of

In [41]:
for i in pop:
    print(i)

maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 'enemy_herohealth')
maximum_minimum(Moves, 'minion_damage', 

In [48]:
pop = toolbox.population(n=100)
for i in pop:
    print(i)

IfThenElse(opGreaterEquals('enemy_herohealth', 30), minimum_minimum(Moves, 'herohealth', 'enemy_number_of_minions'), minimum_maximum(Moves, 'herohealth', 'potential_damage'))
IfThenElse(opGreaterEquals('enemy_number_of_minions', 3), minimum_maximum(Moves, 'enemy_minion_damage', 'enemy_number_of_minions'), maximum_minimum(Moves, 'enemy_number_of_minions', 'enemy_number_of_minions'))
IfThenElse(opGreaterEquals('enemy_number_of_minions', 0), minimum_maximum(Moves, 'mana', 'enemy_minions'), minimum_minimum(Moves, 'potential_damage', 'minions'))
IfThenElse(opGreaterEquals('mana', 20), minimum_minimum(Moves, 'herohealth', 'potential_damage'), maximum_maximum(Moves, 'minions', 'enemy_number_of_minions'))
IfThenElse(opLessEquals('minions', 6), maximum_minimum(Moves, 'potential_damage', 'enemy_minions'), minimum_maximum(Moves, 'enemy_minions', 'potential_damage'))
IfThenElse(opLessEqualsStr('minions', 'enemy_number_of_minions'), minimum_minimum(Moves, 'minion_damage', 'enemy_minion_damage'), ma

In [ ]:
#ind = gp.PrimitiveTree.from_string("IfThenElse(opLessEquals('enemy_number_of_minions', 15), minimum_minimum(Moves, 'minion_damage', 'enemy_number_of_minions'), maximum_minimum(Moves, 'potential_damage', 'enemy_minion_damage'))", pset)
ind = gp.PrimitiveTree.from_string("maximum_minimum(Moves, 'number_of_minions', 'enemy_herohealth')", pset)
#print(evaluateIndv(ind, opponent=toolbox.opponent(), n=100))
for i in range(0,10):
    print(evaluateIndv(ind, opponent=toolbox.opponent(), n=250))

(2.3984,)
(2.5084,)
(2.2852,)
(2.2676,)
(2.4776,)
(2.3708,)
(2.3084000000000002,)
(2.3168,)
(2.2716000000000003,)


In [336]:
#ind = toolbox.population(n=1)[0]
ind = gp.PrimitiveTree.from_string("IfThenElse(and_(opGreaterEquals('minion_damage', 21), opLessEqualsStr('enemy_minion_damage', 'enemy_herohealth')), minimum_maximum(Moves, 'minions', 'herohealth'), IfThenElse(opLessEqualsStr('potential_damage', 'number_of_minions'), maximum_maximum(Moves, 'enemy_herohealth', 'minions'), minimum_maximum(Moves, 'herohealth', 'enemy_number_of_minions')))", pset)
#ind = gp.PrimitiveTree.from_string("IfThenElse(opLessEquals('enemy_number_of_minions', 15), minimum_minimum(Moves, 'minion_damage', 'enemy_number_of_minions'), maximum_minimum(Moves, 'potential_damage', 'enemy_minion_damage'))", pset)
#subtree_slice = ind.searchSubtree(1)
#subtree = ind[subtree_slice]
#print(subtree)
#print(ind.index(subtree[0]))
#print(ind.index(subtree[len(subtree)-1]))
#for i in ind:
#    print(i)

print(ind)
#print(ind.height)
#toolbox.compile(ind)([])

print("_______________")

#child = createMutationIndv(ind)
new_ind = toolbox.mutate(ind, pset=pset)
print(new_ind)
#print(new_ind.height)
#for i in new_ind:
#    print(i)

#global_gstate = MoveList()
#global_gstate.minions = 0
#toolbox.compile(ind)([])

IfThenElse(and_(opGreaterEquals('minion_damage', 21), opLessEqualsStr('enemy_minion_damage', 'enemy_herohealth')), minimum_maximum(Moves, 'minions', 'herohealth'), IfThenElse(opLessEqualsStr('potential_damage', 'number_of_minions'), maximum_maximum(Moves, 'enemy_herohealth', 'minions'), minimum_maximum(Moves, 'herohealth', 'enemy_number_of_minions')))
_______________
Chosen: and_
New: opGreaterEqualsStr
temp: ['IfThenElse', 'opGreaterEqualsStr', 'potential_damage', 'number_of_minions', 'minimum_maximum', 'ARG0', 'minions', 'herohealth', 'IfThenElse', 'opLessEqualsStr', 'potential_damage', 'number_of_minions', 'maximum_maximum', 'ARG0', 'enemy_herohealth', 'minions', 'minimum_maximum', 'ARG0', 'herohealth', 'enemy_number_of_minions']
IfThenElse(opGreaterEqualsStr('potential_damage', 'number_of_minions'), minimum_maximum(Moves, 'minions', 'herohealth'), IfThenElse(opLessEqualsStr('potential_damage', 'number_of_minions'), maximum_maximum(Moves, 'enemy_herohealth', 'minions'), minimum_maxi

In [80]:
for i in ind:
    print(i)
print(ind[0].args)

print(ind[11].name)
print(ind[11].ret)
print(ind[11].value)
print(len(ind))

[<class 'bool'>, <class '__main__.MoveListFinal'>, <class '__main__.MoveListFinal'>]
potential_damage
<class 'str'>
potential_damage
12
